In [27]:
import random as rnd
import numpy as np
import math

class Problem:
    def __init__(self):
        self.dimension = 11
        self.original_costs = [
            [0, 5, 10, 15, 7, 12, 20, 8, 14, 10, 9],
            [5, 0, 6, 9, 5, 8, 14, 4, 7, 5, 6],
            [10, 6, 0, 8, 4, 6, 13, 7, 5, 8, 9],
            [15, 9, 8, 0, 7, 4, 10, 5, 3, 6, 12],
            [7, 5, 4, 7, 0, 3, 9, 6, 4, 5, 8],
            [12, 8, 6, 4, 3, 0, 9, 7, 3, 5, 6],
            [20, 14, 13, 10, 9, 7, 0, 6, 5, 8, 10],
            [8, 4, 7, 5, 6, 3, 6, 0, 2, 5, 7],
            [14, 7, 5, 3, 4, 5, 5, 2, 0, 3, 8],
            [10, 5, 8, 6, 5, 3, 8, 5, 3, 0, 4],
            [9, 6, 9, 12, 8, 9, 10, 7, 8, 4, 0]
        ]
        self.costs = np.array(self.original_costs, dtype=float)
        self.num_vehicles = 1
        self.depot = 0

#Verificar que no se salga más de una vez de un mismo lugar        
    def check(self, solution):
        status = True
        i = 0
        while i < self.dimension and status:
            status = sum(solution[i]) == 1
            i += 1
        i = 0
        while i < self.dimension and status:
            status = sum(j[i] for j in solution) == 1
            i += 1

#algoritmo Bellman-Ford para detectar ciclos en la solución
    def has_cycle(self, solution):
        n = len(solution)
        dist = [float('inf')] * n
        pred = [None] * n
        dist[0] = 0

        for _ in range(n - 1):
            for u in range(n):
                for v in range(n):
                    if u != v and dist[v] > dist[u] + self.costs[solution[u]][solution[v]]:
                        dist[v] = dist[u] + self.costs[solution[u]][solution[v]]
                        pred[v] = u

        for u in range(n):
            for v in range(n):
                if u != v and dist[v] > dist[u] + self.costs[solution[u]][solution[v]]:
                    return True

        return False

class Sparrow:
    def __init__(self, problem):
        self.problem = problem
        self.dimension = problem.dimension
        self.position = np.random.permutation(self.dimension - 1) + 1
        self.position = np.concatenate(([0], self.position, [0]))
        self.velocity = np.zeros(self.dimension, dtype=float)

    def is_feasible(self):
        return len(set(self.position)) == self.dimension and not self.problem.has_cycle(self.position)

    def fitness(self):
        cost = 0
        prev_location = self.problem.depot
        for location in self.position:
            cost += self.problem.costs[prev_location][location]
            prev_location = location
        cost += self.problem.costs[prev_location][self.problem.depot]
        return cost

    def isbetterthan(self, o):
        return self.fitness() < o.fitness()

    def move(self, swarm, iter, R2, ST, alpha, maxiter):
        num_sparrows = len(swarm)
        PD = 0.2 #% de productores
        num_producers = int(PD * num_sparrows)
        num_scroungers = num_sparrows - num_producers  #numero de merodeadores

        for i in range(num_producers):
            swarm[i].update_producer(R2, ST, iter, maxiter, alpha)
            swarm[i].make_feasible()

        X_best, X_worst = self.find_best_worst(swarm)
        for i in range(num_producers, num_sparrows):
            swarm[i].update_scrounger(i, num_sparrows, X_best, X_worst)
            swarm[i].make_feasible()

        f_best = self.fitness()
        f_worst = swarm[-1].fitness()
        self.update_aware(X_best, X_worst, f_best, f_worst)
        self.make_feasible()

    def find_best_worst(self, swarm):
        best_fitness = float('inf')
        worst_fitness = float('-inf')
        X_best, X_worst = None, None
        for sparrow in swarm:
            fitness = sparrow.fitness()
            if fitness < best_fitness:
                best_fitness = fitness
                X_best = sparrow.position.copy()
            if fitness > worst_fitness:
                worst_fitness = fitness
                X_worst = sparrow.position.copy()
        return X_best, X_worst

    def make_feasible(self):
        position = np.random.permutation(self.dimension - 1) + 1
        self.position = np.concatenate(([0], position, [0]))

    def update_producer(self, R2, ST, iter, maxiter, alpha):
        if R2 < ST:
            self.position = (self.position * np.exp(-iter / (alpha * maxiter))).astype(int)
        else:
            self.position = (self.position + np.random.normal(0, 1, size=self.position.shape)).astype(int)

    def update_scrounger(self, i, num_sparrows, X_best, X_worst):
        if i > num_sparrows / 2:
            self.position = (np.random.normal(0, 1) * np.exp((X_worst - self.position) / i**2)).astype(int)
        else:
            self.position = (X_best + np.abs(self.position - X_best) * np.random.choice([-1, 1], size=self.position.shape)).astype(int)

    def update_aware(self, X_best, X_worst, f_best, f_worst):
        if self.fitness() > f_best:
            self.position = (X_best + np.random.normal(0, 1) * np.abs(self.position - X_best)).astype(int)
        else:
            self.position = (self.position + np.random.uniform(-1, 1) * np.abs(self.position - X_worst) / (self.fitness() - f_worst + 1e-8)).astype(int)

    def copy(self, a):
        self.position = a.position.copy()
        self.velocity = a.velocity.copy()

class SO:
    def __init__(self):
        self.problem = Problem()
        self.maxiter = 1000 # Definir el número máximo de iteraciones
        self.nsparrows = 20  # Definir el número de gorriones en el enjambre
        self.R2 = 0.8  # Definir el valor de R2 (parámetro de control)
        self.ST = 0.6 # Definir el valor de ST (umbral de seguridad)
        self.alpha = 1.0  # Definir el valor de alpha (parámetro de control)
        self.swarm = []
        self.g = Sparrow(self.problem)

    def solve(self):
        self.initrand()
        self.evolve()
        print("Best solution found:")
        print(self.g.position)

        total_cost = self.calculate_total_cost(self.g.position, self.problem.original_costs)
        print("Total cost of the best solution:", total_cost)

    def initrand(self):
        for i in range(self.nsparrows):
            while True:
                s = Sparrow(self.problem)
                if s.is_feasible():
                    break
            self.swarm.append(s)
        self.g.copy(self.swarm[0])
        for i in range(1, self.nsparrows):
            if self.swarm[i].isbetterthan(self.g):
                self.g.copy(self.swarm[i])

    def evolve(self):
        t = 1
        best_fitness = self.g.fitness()
        while t <= self.maxiter:
            for i in range(self.nsparrows):
                while True:
                    self.swarm[i].move(self.swarm, t, self.R2, self.ST, self.alpha, self.maxiter)
                    if self.swarm[i].is_feasible():
                        break
                if self.swarm[i].fitness() < best_fitness:
                    self.g.copy(self.swarm[i])
                    best_fitness = self.swarm[i].fitness()
        
            # Agregar la funcionalidad de los gorriones conscientes del peligro
            SD = 0.1
            num_aware = int(SD * self.nsparrows)
            for i in range(num_aware):
                self.swarm[i].update_aware(self.g.position, self.swarm[-1].position, best_fitness, self.swarm[-1].fitness())
                self.swarm[i].make_feasible()
        
            t += 1

    def calculate_total_cost(self, solution, cost_matrix):
        total_cost = 0
        for i in range(len(solution) - 1):
            total_cost += cost_matrix[solution[i]][solution[i+1]]
        return total_cost

if __name__ == "__main__":
    np.random.seed(2024)
    SO().solve()

Best solution found:
[ 0  1 10  9  5  3  8  6  7  2  4  0]
Total cost of the best solution: 54
